In [1]:
import sys; sys.path.append("../..")
from EncoderDecoderShareVAE import EncoderDecoderShareVAE
import NewsVAEArguments
import torch
from transformers.generation_utils import top_k_top_p_filtering
from NewsData import NewsData
import numpy as np
import copy
import torch.nn.functional as F
from trainNewsVAE import load_from_checkpoint
from utils import transfer_batch_to_device
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
# from pytorch_lightning import seed_everything; seed_everything(0)

# Check if on server
if torch.cuda.is_available():
    print("Using GPU :-)...")
    device = "cuda"
else:
    print("Warning: using CPU!")
    device = "cpu"
    
# Overwrite some arguments
args = NewsVAEArguments.preprare_parser(jupyter=True, print_settings=False)
args.ddp = False
args.n_gpus = 1

import multiprocessing; print("Number of CPU cores: {}".format(multiprocessing.cpu_count()))

Using GPU :-)...
Number of CPU cores: 12


In [2]:
# Get data
data = NewsData(args.dataset_name, args.tokenizer_name,
                batch_size=args.batch_size, num_workers=args.num_workers,
                pin_memory=(device=="cuda"), debug=args.debug_data,
                debug_data_len=args.debug_data_len, max_seq_len=args.max_seq_len, device=device)

In [3]:
# Get model architecture
VAE_model = EncoderDecoderShareVAE(args, args.base_checkpoint_name, do_tie_weights=args.do_tie_weights).to(device)

# # Load weights
beta_vae_path = "/home/cbarkhof/code-thesis/NewsVAE/Runs/18NOV-BETA-VAE-run-2020-11-18-12:36:55/checkpoint-50000.pth"
mmd_vae_path = "/home/cbarkhof/code-thesis/NewsVAE/Runs/18NOV-MMD-VAE-run-2020-11-18-12:36:55/checkpoint-50000.pth"
ae_path = "/home/cbarkhof/code-thesis/NewsVAE/Runs/23NOV-AUTOENCODER-run-2020-11-23-18:36:12/checkpoint-50000.pth"

# Need to make a deepcopy otherwise the weights are assigned to the original VAE_model which leads to confusing results :O
_, _, beta_vae_model, _, _, _, _ = load_from_checkpoint(copy.deepcopy(VAE_model), args, path=beta_vae_path)
_, _, mmd_vae_model, _, _, _, _ = load_from_checkpoint(copy.deepcopy(VAE_model), args, path=mmd_vae_path)
_, _, ae_model, _, _, _, _ = load_from_checkpoint(copy.deepcopy(VAE_model), args, path=ae_path)

del VAE_model
beta_vae_model.eval(); mmd_vae_model.eval(); ae_model.eval();

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of VAE_Decoder_RobertaForCausalLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<class 'VAE_Decoder_Roberta.VAE_Decoder_RobertaModel'> and <class 'transformers.modeling_roberta.RobertaModel'> are not equal. In this case make sure that all encoder weights are correctly initialized. 
The following encoder weights were not tied to the decoder ['roberta/pooler']
Loading VAE_model, optimizer and scheduler from /home/cbarkhof/code-thesis/NewsVAE/Runs/18NOV-BETA-VAE-run-2020-11-18-12:36:55/checkpoint-50000.pth
first removing module string from checkpoint
Checkpoint global_step: 50000, epoch: 33, best_valid_loss: 3.116105151176453
Loading VAE_model, optimizer and scheduler from /home/cbarkhof/code-thesis/NewsVAE/Runs/18NOV-MMD-VAE-run-2020-11-18-12:36:55/checkpoint-50000.pth
first removing module string from checkpoint
Checkpoint global_step: 50000, epoch: 33, best_valid_loss: 2.233636903762817
Loading VAE_model, optimizer and scheduler from /home/cbarkhof/code-thesis/NewsVAE/Runs/23NOV-AUTOENCODER-run-2020-11-23-18:36:12/checkpoint-50000.pth
first removing module string 

In [4]:
def sample_from_prior(latent_size=768, n_samples=8):
    loc = torch.zeros(latent_size, device='cuda:0')
    scale = torch.ones(latent_size, device='cuda:0')
    prior_dist = torch.distributions.normal.Normal(loc, scale)
    samples = prior_dist.sample((n_samples,))
    return samples
    
def sample_text(model, tokenizer, args, n_samples=8, max_seq_len=64, nucleus_sampling=True):
    with torch.no_grad():
        latent_z = sample_from_prior(model.latent_size, n_samples=n_samples)
        generated = auto_regressive_decode(model, latent_z, tokenizer, args, max_seq_len=max_seq_len, nucleus_sampling=nucleus_sampling)
        generated_text = batch_decode(generated, tokenizer)
        return generated_text

def batch_decode(batch_of_samples, tokenizer):    
    return [tokenizer.decode(a.tolist()) for a in batch_of_samples]

def auto_regressive_decode(model, latent_z, tokenizer, args, max_seq_len=32, nucleus_sampling=False, 
                           temperature=1.0, top_k=0, top_p=0.9):
    batch_size = latent_z.shape[0]

    # Add <s> and </s>
    generated_so_far = torch.tensor([[data.tokenizer.bos_token_id, data.tokenizer.eos_token_id] for _ in range(batch_size)])
    generated_so_far = generated_so_far.cuda(0)

    for i in range(max_seq_len):        
        # Forward the decoder
        decoder_outs = model.decoder(input_ids=generated_so_far, attention_mask=None,
                                    latent_z=latent_z, labels=None,
                                    add_latent_via_embeddings=args.add_latent_via_embeddings,
                                    add_latent_via_memory=args.add_latent_via_memory,
                                    return_cross_entropy=False, return_predictions=True,
                                    return_exact_match_acc=False)

        if nucleus_sampling:
            # The forward of the decoder already cuts of the prediction for the last token
            logits = decoder_outs["logits"][:, -1, :]

            # Temperature (higher temperature => more likely to sample low probability tokens)
            if temperature != 1.0:
                scores = scores / temperature

            # Top-p/top-k filtering
            next_token_logscores = top_k_top_p_filtering(logits, top_k=top_k, top_p=top_p)

            # Sample
            probs = F.softmax(next_token_logscores, dim=-1)
            new_preds = torch.multinomial(probs, num_samples=1).squeeze(1)
        else:
            new_preds = decoder_outs['predictions'][:, -1] # argmax predictions

        generated_so_far = torch.cat((generated_so_far[:, :-1], new_preds.unsqueeze(1), generated_so_far[:, -1].unsqueeze(1)), dim=1)
    return generated_so_far

def encode(model, input_batch, deterministic=False):
    encoder_outs = model.encoder(input_ids=input_batch['input_ids'], attention_mask=input_batch['attention_mask'])
    latent_z, _, _ = model.connect_encoder_decoder(encoder_outs.pooler_output, deterministic=deterministic)
    
    return latent_z

def encode_auto_regressive_decode(model, input_batch, tokenizer, args, max_seq_len=64, nucleus_sampling=True):
    with torch.no_grad():
        latent_z = encode(model, input_batch)
        generated = auto_regressive_decode(model, latent_z, tokenizer, args, max_seq_len=max_seq_len, nucleus_sampling=nucleus_sampling)
        generated_text = batch_decode(generated, tokenizer)
        
        return generated_text

# Text reconstructions

In [7]:
# max_batches = 5
# results = {'aaNotionColumn': [f"aaNotionValue_{i}" for i in range(max_batches * args.batch_size)],
#            'original': [],
#            'mmd-vae': [],
#            'beta-vae': [],
#            'ae-model':[],
#           }


# for batch_i, input_batch in enumerate(data.val_dataloader(shuffle=True)):
#     print(f"Batch {batch_i}", end="\r")
#     input_batch = transfer_batch_to_device(input_batch, 0)
#     original_text = batch_decode(input_batch['input_ids'], data.tokenizer)
#     results['original'].extend(original_text)
    
#     for model, model_name in zip([mmd_vae_model, beta_vae_model, ae_model], ['mmd-vae', 'beta-vae', 'ae-model']):
#         generated_text = encode_auto_regressive_decode(model, input_batch, data.tokenizer, 
#                                                        args, max_seq_len=64, nucleus_sampling=True)
#         results[model_name].extend(generated_text)

#     if batch_i + 1 == max_batches: break
        
# df = pd.DataFrame(results)
# df.to_csv('text_reconstructions.csv')

# Sampling

In [8]:
# # Sample
# n_samples = 1000
# batch_size = 10

# prior_samples = sample_from_prior(latent_size=768, n_samples=n_samples)

# encoded_samples_beta = []
# encoded_samples_mmd = []
# encoded_samples_ae = []

# with torch.no_grad():
#     assert n_samples % batch_size == 0, "n_samples must be exactly divisible by the batch size!"
#     max_batches = int(n_samples / batch_size)
    
#     print("max_batches:", max_batches)
    
#     for batch_i, input_batch in enumerate(data.val_dataloader(shuffle=True, batch_size=batch_size)):
#         if batch_i % 10 == 0: print("Batch {:2d}/{}".format(batch_i, max_batches), end='\r')

#         input_batch = transfer_batch_to_device(input_batch, 0)
        
#         encoded_samples_beta.append(encode(beta_vae_model, input_batch, deterministic=False))
#         encoded_samples_mmd.append(encode(mmd_vae_model, input_batch, deterministic=False))
#         encoded_samples_ae.append(encode(ae_model, input_batch, deterministic=False))

#         if batch_i + 1 == max_batches: break
        
# encoded_samples_beta = torch.cat(encoded_samples_beta)
# encoded_samples_mmd = torch.cat(encoded_samples_mmd)
# encoded_samples_ae = torch.cat(encoded_samples_ae)

# print(prior_samples.shape, encoded_samples_beta.shape, encoded_samples_mmd.shape, encoded_samples_ae.shape)

# # Reduce dimensionality PCA -> TSNE
# all_samples = torch.cat((prior_samples, encoded_samples_beta, encoded_samples_mmd, encoded_samples_ae))
# all_samples_pca = PCA(n_components=50).fit_transform(all_samples.cpu().numpy())
# all_samples_tsne_2dim = TSNE(n_components=2, n_iter=500, perplexity=20).fit_transform(all_samples_pca)

# # Plot
# plt.scatter(all_samples_tsne_2dim[:1000, 0], all_samples_tsne_2dim[:1000, 1], label='Prior', alpha=0.3, color='green')
# plt.scatter(all_samples_tsne_2dim[1000:2000, 0], all_samples_tsne_2dim[1000:2000, 1], label='Encoded Beta-VAE', alpha=0.3, color='blue')
# plt.scatter(all_samples_tsne_2dim[2000:3000, 0], all_samples_tsne_2dim[2000:3000, 1], label='Encoded MMD-VAE', alpha=0.3, color='pink')
# plt.scatter(all_samples_tsne_2dim[3000:, 0], all_samples_tsne_2dim[3000:, 1], label='Encoded AE', alpha=0.3, color='orange')
# plt.legend()
# plt.savefig("latent_space_plt.png", dpi=300)
# plt.show()

In [11]:
n_samples = 10
sampling_results = {'aaNotionColumn': [f"aaNotionValue_{i}" for i in range(n_samples)]}

for model, name in zip([mmd_vae_model, beta_vae_model, ae_model], ['mmd-vae', 'beta-vae', 'ae-model']):
    print('Samples from:', name)
    gen_text = sample_text(model, data.tokenizer, args, n_samples=n_samples, max_seq_len=64, nucleus_sampling=True)
    sampling_results[name] = gen_text
#     for i, t in enumerate(gen_text):
#         print(i)
#         print(t)
#         print()
#     print("-"*30)
                    
df = pd.DataFrame(sampling_results)
display(df)
df.to_csv('sampling.csv')

Samples from: mmd-vae
0
<s>Traits forfeishing in toing not in to to to by being in au in in was was noting in to in when to to Rebecca di didnne has the ending to being ownership in the Juan Juan is hasu has been voted Elizabeth has Peter has to the the the Kenne to the the</s></s></s>

1
<s>By and and and and and Rick and. combined.. Shane Shane Shane.'s Sunday Thursday. Wednesday Thursday Wednesday. Monday Monday Wednesday. Sunday Thursday Wednesday Wednesday Thursday. Mick.. Tom. He Foley.. Wednesday. Wednesday. Duffy. Tony.. Friday has has...</s>. Rey has</s></s></s>

2
<s> monthsil,,,il-,il,il-il----az,ilil,il,ililacheache-ilil world (:: (:----to::, to to for to:.:,,..- in to.-</s></s></s></s>

3
<s>LLE David David " -- AL D "" "2 that as a David would could would take alers he if as he had had had a a having had that a had'd an a a. an had made an an.. an. the that the had.. a.. had a.</s></s></s>

4
<s> In-�-es,As-ysys--giesys)--igys:,. and-ies�,-ies), – and, and?.,,, and-w-�, a

,aaNotionColumn,mmd-vae,beta-vae,ae-model
0,aaNotionValue_0,<s>Traits forfeishing in toing not in to to to...,<s>By. Daily Mail Reporter. PUBLISHED:. 03:17 ...,<s>sim not then that that another but not. one...
1,aaNotionValue_1,<s>By and and and and and Rick and. combined.....,"<s>By. Leon Watson. PUBLISHED:. 07:49 EST, 15 ...","<s>A220.220IT:. C::. $--245, positive-cent. fa..."
2,aaNotionValue_2,"<s> monthsil,,,il-,il,il-il----az,ilil,il,ilil...",<s>(CNN) -- Ten Turkish people held by al Qae...,<s>kekekekekekekekekekekekekekekekekekekekekek...
3,aaNotionValue_3,"<s>LLE David David "" -- AL D """" ""2 that as a D...",<s>By. Sam Webb. A kangaroo who lived in a Sib...,<s>WASHINGTON (CNN)))))))))))) (10) 1) in Mayo...
4,aaNotionValue_4,"<s> In-�-es,As-ysys--giesys)--igys:,. and-ies�...","<s>LONDON, England (CNN) -- If any question ab...","<s>GR,SW,SW,BR-KN, to to-evev approved,, polic..."
5,aaNotionValue_5,<s> Aviation outside his association's's has h...,<s>(CNN) -- Rolling Stone magazine is offering...,<s>CLIAED KarlOGEE HARTENHOGERS FOR CECKE FEDE...
6,aaNotionValue_6,<s>(gewew---the--t/-thethe-mmm. business-ains ...,"<s>By. Mark Duell. PUBLISHED:. 11:12 EST, 15 A...",<s>By#aminaminte. -- 15). more to to about. Aa...
7,aaNotionValue_7,"<s>TheousousousousThe andous and has the,a has...",<s>(CNN) -- At the crossroads of European and ...,"<s> Alto. Reading,, and.</s>ow, the andow, of ..."
8,aaNotionValue_8,<s>(ingiana)'ni (ani)). (13)))))inging. (())))...,<s>By. Daily Mail Reporter. PUBLISHED:. 03:38 ...,<s>Uill ( ( ( (win ( ($ ($ ($ ($ (R--/( (69) 8...
9,aaNotionValue_9,"<s>.. Is. Is...,?.., Is...................,.,...","<s>By. Rachel Quigley. PUBLISHED:. 08:42 EST, ...",<s> lastyawn hasouted alacedaced on a spot and...


In [ ]:
.to_csv("sampling.csv")